In [1]:
%load_ext autoreload
%autoreload 2

import asyncio, copy, os, pickle, socket, sys, time
from functools import partial
from multiprocessing import Pool, Process
from pathlib import Path
from tqdm import tqdm

import torch
from torch import optim
from torch.utils.tensorboard import SummaryWriter

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import agg, data, fl, log, nn, poison, resnet, sim
from cfgs.fedargs import *

In [2]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", "poison.log")
#log.init("debug", "poison.log")

In [3]:
# Make changes as required by looking at the cfgs/fedargs
fedargs.tb = SummaryWriter('../../out/runs/federated/FLTrust/mn-sine-500*2-dot-5+1-in', comment="posion")
fedargs.agg_rule = agg.Rule.FLTrust
#fedargs.dataset = "cifar10" # for cifar-10,"fmnist" for fmnist
#fedargs.model = resnet.ResNet18() # for cifar10
mal_clients = [c for c in range(20)]
FLTrust["is"] = True
FLTrust["proxy"]["is"] = False
label_flip_attack["is"] = False
label_flip_attack["func"] = poison.label_flip_next
set_lfa_labels()
backdoor_attack["is"] = False
layer_replacement_attack["is"] = False
cosine_attack["is"] = True
#cosine_attack["kn"] = poison.Knowledge.FN
#cosine_attack["kn"] = poison.Knowledge.PN
fang_attack["is"] = False
#fang_attack["kn"] = poison.Knowledge.FN
lie_attack["is"] = False
#lie_attack["kn"] = poison.Knowledge.FN
sota_attack["is"] = False
#sota_attack["kn"] = poison.Knowledge.FN
#dev_type: unit_vec
#sota_attack["dev_type"] = "sign"
#sota_attack["dev_type"] = "std"
#func: sota_agnostic_min_max
#sota_attack["func"] = poison.sota_agnostic_min_sum
#sota_attack["func"] = poison.sota_agr_tailored_trmean
sybil_attack["is"] = False

In [4]:
# Device settings
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [5]:
# Prepare clients
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]

In [6]:
# Initialize Global and Client models
global_model = copy.deepcopy(fedargs.model)
# Load Data to clients
train_data, test_data = data.load_dataset(fedargs.dataset)

<h2>FLTrust</h2>

In [7]:
if FLTrust["is"]:
    train_data, FLTrust["data"] = data.random_split(train_data, FLTrust["ratio"])
    FLTrust["loader"] = torch.utils.data.DataLoader(FLTrust["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)
    
    if FLTrust["proxy"]["is"]:
        FLTrust["data"], FLTrust["proxy"]["data"] = data.random_split(FLTrust["data"], FLTrust["proxy"]["ratio"])
        FLTrust["loader"] = torch.utils.data.DataLoader(FLTrust["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)
        FLTrust["proxy"]["loader"] = torch.utils.data.DataLoader(FLTrust["proxy"]["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

<h2>Prepare a backdoored loader for test</h2>

In [8]:
if backdoor_attack["is"]:
    train_data, backdoor_attack["data"] = data.random_split(train_data, backdoor_attack["ratio"])
    backdoor_attack["data"] = poison.insert_trojan(backdoor_attack["data"],
                                                   backdoor_attack["target_label"],
                                                   backdoor_attack["trojan_func"], 1)
    backdoor_attack["loader"] = torch.utils.data.DataLoader(backdoor_attack["data"], batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

<h2>Load client's data</h2>

In [9]:
clients_data = data.split_data(train_data, clients)

<h2>Label Flip Attack</h2>

In [10]:
if label_flip_attack["is"]:
    for client in mal_clients:
        clients_data[clients[client]] = label_flip_attack["func"](clients_data[clients[client]],
                                                                  label_flip_attack["labels"],
                                                                  label_flip_attack["percent"])

<h2>Backdoor Attack</h2>

In [11]:
if backdoor_attack["is"]:
    for client in mal_clients:
        clients_data[clients[client]] = poison.insert_trojan(clients_data[clients[client]],
                                                             backdoor_attack["target_label"],
                                                             backdoor_attack["trojan_func"], 0.5)

In [12]:
client_train_loaders, _ = data.load_client_data(clients_data, fedargs.client_batch_size, None, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

client_details = {
        client: {"train_loader": client_train_loaders[client],
                 "model":  copy.deepcopy(global_model),
                 "model_update": None}
        for client in clients
    }

In [13]:
def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, model, train_loader, fedargs, device):
    # Train
    model_update, model, loss = fl.train_model(model, train_loader, 
                                            fedargs.learning_rate,
                                            fedargs.weight_decay,
                                            fedargs.local_rounds, device)

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model_update, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))
    
    return model_update

In [ ]:
import time
start_time = time.time()
    
# Federated Training
for _epoch in tqdm(range(fedargs.epochs)):

    epoch = _epoch + 1
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))

    # Gloabal Model Update
    if epoch > 1:
        # For Tmean and FLTrust, not impacts others as of now
        avgargs = {"beta": len(mal_clients), 
                   "base_model_update": global_model_update if FLTrust["is"] else None,
                   "base_norm": True}
        
        # Average
        global_model = fl.federated_avg(client_model_updates, global_model, fedargs.agg_rule, **avgargs)
        log.modeldebug(global_model, "Epoch {} of {} : Server Update".format(epoch, fedargs.epochs))

        # Test
        global_test_output = fl.evaluate(global_model, test_loader, device, label_flip_attack["labels"])
        fedargs.tb.add_scalar("Gloabl Accuracy/", global_test_output["accuracy"], epoch)
        fedargs.tb.add_scalar("Gloabl Test Loss/", global_test_output["test_loss"], epoch)
        if "attack" in global_test_output:
            if "attack_success_rate" in global_test_output["attack"]:
                fedargs.tb.add_scalar("Attack Success Rate/", global_test_output["attack"]["attack_success_rate"], epoch)
            if "misclassification_rate" in global_test_output["attack"]:
                fedargs.tb.add_scalar("Misclassification Rate/", global_test_output["attack"]["misclassification_rate"], epoch)
        if backdoor_attack["is"]:
            backdoor_test_output = fl.backdoor_test(global_model, backdoor_attack["loader"], device,
                                                                backdoor_attack["target_label"])
            fedargs.tb.add_scalar("Backdoor Success Rate/", backdoor_test_output["accuracy"], epoch)
            log.jsoninfo(backdoor_test_output, "Backdoor Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
        log.jsoninfo(global_test_output, "Global Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
    
        # Update client models
        for client in clients:
            client_details[client]['model'] = copy.deepcopy(global_model)

    # Clients
    tasks = [process(client, epoch, client_details[client]['model'],
                     client_details[client]['train_loader'],
                     fedargs, device) for client in clients]
    try:
        updates = fedargs.loop.run_until_complete(asyncio.gather(*tasks))
    except KeyboardInterrupt as e:
        print("Caught keyboard interrupt. Canceling tasks...")
        tasks.cancel()
        fedargs.loop.run_forever()
        tasks.exception()

    for client, update in zip(clients, updates):
        client_details[client]['model_update'] = update
    client_model_updates = {client: details["model_update"] for client, details in client_details.items()}
    
    # Fang attack
    if fang_attack["is"]:
        client_model_updates = fang_attack["func"](client_model_updates, len(mal_clients), fang_attack["kn"])
        
    # LIE attack
    if lie_attack["is"]:
        client_model_updates = lie_attack["func"](client_model_updates, len(mal_clients), lie_attack["kn"])
        
    # SOTA attack
    if sota_attack["is"]:
        client_model_updates = sota_attack["func"](client_model_updates, len(mal_clients), 
                                                   sota_attack["kn"], sota_attack["dev_type"])    
    
    if FLTrust["is"]:
        global_model_update, _, _ = fl.train_model(global_model, FLTrust["loader"],
                                                fedargs.learning_rate,
                                                fedargs.weight_decay,
                                                fedargs.local_rounds, device)

        # For Attacks related to FLTrust
        base_model_update = global_model_update
        if FLTrust["proxy"]["is"]:
            base_model_update, _, _ = fl.train_model(global_model, FLTrust["proxy"]["loader"],
                                                  fedargs.learning_rate,
                                                  fedargs.weight_decay,
                                                  fedargs.local_rounds, device)
            
        if layer_replacement_attack["is"]:
            for client in mal_clients:
                client_details[clients[client]]['model_update'] = layer_replacement_attack["func"](base_model_update,
                                                                                                   client_details[clients[client]]['model_update'],
                                                                                                   layer_replacement_attack["layers"])

        # For cosine attack, Malicious Clients
        if cosine_attack["is"]:
            p_models, params_changed = cosine_attack["func"](base_model_update, cosine_attack["args"], epoch,
                                                             client_model_updates, len(mal_clients), cosine_attack["kn"])
            
            for client, p_model in enumerate(p_models):
                client_details[clients[client]]['model_update'] = p_model 

            #plot params changed for only one client
            fedargs.tb.add_scalar("Params Changed for Cosine Attack/", params_changed, epoch)

        # For sybil attack, Malicious Clients
        if sybil_attack["is"]:
            for client in mal_clients:
                client_details[clients[client]]['model_update'] = base_model_update
                
        # again pair, as changed during attack
        client_model_updates = {client: details["model_update"] for client, details in client_details.items()}

print(time.time() - start_time)

  2%|▏         | 1/50 [01:49<1:29:14, 109.28s/it]2021-09-24 08:36:14,710 - <ipython-input-14-fcdf132f0c06>::<module>(l:8) : Federated Training Epoch 2 of 50 [MainProcess : MainThread (INFO)]
2021-09-24 08:36:20,305 - /home/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:101) : Cosine Score [0.43217033, 0.48342502, 0.3942573, 0.44002193, 0.42211074, 0.4260563, 0.38087636, 0.42049345, 0.44744653, 0.42467517, 0.4195224, 0.4182254, 0.42542532, 0.40758356, 0.4564483, 0.4329066, 0.41295826, 0.41139016, 0.45635197, 0.4238479, 0.42564017, 0.41177174, 0.4338222, 0.44378185, 0.4261276, 0.44281626, 0.44813845, 0.44994587, 0.43117225, 0.43070728, 0.44642827, 0.43355915, 0.44438007, 0.4279856, 0.43417254, 0.43315634, 0.44123888, 0.44454527, 0.42119247, 0.43679398, 0.42827383, 0.4476797, 0.43699366, 0.41830012, 0.43319082, 0.45263863, 0.4214589, 0.45287573, 0.42364627, 0.4485283] [MainProcess : MainThread (INFO)]
2021-09-24 08:36:20,328 - /home/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:102

2021-09-24 08:44:16,395 - /home/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:102) : FLTrust Score [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003421787, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15521528, 0.15471648, 0.14908335, 0.15185216, 0.15362448, 0.15436402, 0.15317644, 0.15564555, 0.14833218, 0.15599713, 0.1499017, 0.1506947, 0.15474908, 0.15501624, 0.1532607, 0.15147302, 0.15379344, 0.15149425, 0.15200691, 0.15147904, 0.15297961, 0.15079898, 0.15430754, 0.14933647, 0.15139765, 0.15552665, 0.1484835, 0.15431327, 0.15648484, 0.15395989] [MainProcess : MainThread (INFO)]
2021-09-24 08:44:22,241 - <ipython-input-14-fcdf132f0c06>::<module>(l:35) : Global Test Outut after Epoch 6 of 50 {
    "accuracy": 80.76,
    "correct": 8076,
    "test_loss": 1.4423915716171265
} [MainProcess : MainThread (INFO)]
 12%|█▏        | 6/50 [11:44<1:26:57, 118.58s/it]2021-09-24 08:46:10,256 - <ipython-input-14-fcdf132f0c06>::<module>(l:8) : Federated Training Epoch 7 of 50 [MainProc

 20%|██        | 10/50 [19:31<1:18:06, 117.16s/it]2021-09-24 08:53:57,231 - <ipython-input-14-fcdf132f0c06>::<module>(l:8) : Federated Training Epoch 11 of 50 [MainProcess : MainThread (INFO)]
2021-09-24 08:54:01,180 - /home/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:101) : Cosine Score [0.2685913, 0.22554635, 0.24895558, 0.23114252, 0.2647805, 0.23424312, 0.23249398, 0.28020957, 0.28518078, 0.2528672, 0.22780676, 0.26377222, 0.25346214, 0.30855307, 0.26475248, 0.26213026, 0.26726755, 0.24050444, 0.24426568, 0.2531263, 0.42141655, 0.42859027, 0.41415715, 0.40910527, 0.41821867, 0.4040363, 0.40278655, 0.41055778, 0.41378754, 0.4135587, 0.41847205, 0.40008622, 0.42550686, 0.42496046, 0.41810808, 0.41106942, 0.40940925, 0.40854567, 0.40691033, 0.41596687, 0.42806712, 0.41558164, 0.4217227, 0.40366074, 0.4214257, 0.4130205, 0.41020733, 0.43046263, 0.41286582, 0.40198073] [MainProcess : MainThread (INFO)]
2021-09-24 08:54:01,193 - /home/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust

2021-09-24 09:01:53,778 - /home/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:102) : FLTrust Score [0.14301226, 0.14713787, 0.13971132, 0.15262613, 0.15687329, 0.16265799, 0.15089989, 0.15482436, 0.16081409, 0.15566397, 0.15356506, 0.15143806, 0.15434666, 0.15834326, 0.15306143, 0.156715, 0.15112172, 0.1566953, 0.16105641, 0.14925994, 0.10729671, 0.1142383, 0.10492481, 0.10642064, 0.10261832, 0.10760511, 0.10738624, 0.112174295, 0.11124906, 0.10738499, 0.108807, 0.10786631, 0.11268523, 0.1172302, 0.10721722, 0.10799671, 0.11191946, 0.10492824, 0.1128689, 0.107539155, 0.11263129, 0.10902674, 0.11286115, 0.1168598, 0.10361575, 0.103361055, 0.10239348, 0.10600437, 0.10884711, 0.10545624] [MainProcess : MainThread (INFO)]
2021-09-24 09:01:59,677 - <ipython-input-14-fcdf132f0c06>::<module>(l:35) : Global Test Outut after Epoch 15 of 50 {
    "accuracy": 86.29,
    "correct": 8629,
    "test_loss": 0.5923381776809692
} [MainProcess : MainThread (INFO)]
 30%|███       | 15/50 [29:20<1:08:

2021-09-24 09:09:54,654 - /home/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:101) : Cosine Score [0.5833197, 0.58667606, 0.5771924, 0.5361121, 0.575298, 0.57383, 0.5861366, 0.60984206, 0.56172556, 0.55381495, 0.5413922, 0.6333503, 0.56937116, 0.61273843, 0.60607225, 0.6738742, 0.58672535, 0.58665895, 0.5694251, 0.55155027, 0.24690232, 0.2563784, 0.2399828, 0.25743413, 0.24800266, 0.23029749, 0.23727697, 0.26262546, 0.25872433, 0.23926426, 0.2614458, 0.25805733, 0.2637647, 0.25714248, 0.25356328, 0.2568727, 0.25312144, 0.26847818, 0.27419564, 0.26160982, 0.2820953, 0.25235406, 0.26290607, 0.26351768, 0.28358024, 0.26253375, 0.22828948, 0.26104838, 0.26206604, 0.2616084] [MainProcess : MainThread (INFO)]
2021-09-24 09:09:54,668 - /home/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:102) : FLTrust Score [0.172224, 0.1760035, 0.17219566, 0.16618726, 0.17141686, 0.16974603, 0.17052568, 0.1725409, 0.16922644, 0.1687113, 0.16790493, 0.17872958, 0.17102037, 0.17305827, 0.17370132, 0.16

<h1> End </h1>